In [1]:
# %pip uninstall -y tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
# %pip install tensorflow

In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)

### utils

In [3]:
def class_mae(y_true, y_pred):
    return K.mean(
        K.abs(
            K.argmax(y_pred, axis=-1) - K.argmax(y_true, axis=-1)
        ),
        axis=-1
    )


In [4]:
# print(keras.__version__)
print(tf.__version__)

2.7.0


In [5]:
# print("Keras version :",keras.__version__)

# Convert model to new tensorflow

In [6]:
with open('json_config/CRNN_config.json') as json_file:
    json_config = json_file.read()

model = tf.compat.v1.keras.models.model_from_json(
    json_config,
    custom_objects={
        'class_mae': class_mae,
        'exp': K.exp,
    }
)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero1 (ZeroPadding2D)       (None, 1, 500, 201)       0         
                                                                 
 conv1 (Conv2D)              (None, 64, 498, 199)      640       
                                                                 
 conv2 (Conv2D)              (None, 32, 496, 197)      18464     
                                                                 
 pool1 (MaxPooling2D)        (None, 32, 165, 65)       0         
                                                                 
 conv3 (Conv2D)              (None, 128, 163, 63)      36992     
                                                                 
 conv4 (Conv2D)              (None, 64, 161, 61)       73792     
                                                                 
 pool2 (MaxPooling2D)        (None, 64, 53, 20)       

In [7]:
model.load_weights("../CountNet/models/CRNN.h5")

# test

In [2]:
model = tf.keras.models.load_model("models/CRNN")
model.compile()

In [3]:
# tf random tensor
tf_random_tensor = tf.random.uniform((1, 1, 500, 201))
print("Input shape", tf_random_tensor.shape)
print("Output shape", model.predict(tf_random_tensor).shape)

Input shape (1, 1, 500, 201)
Output shape (1, 11)


# ONNX